In [0]:
from pyspark import SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType
from pyspark.sql.functions import col, lead,lag,when
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("app").master("local[2]").getOrCreate()

In [0]:
schema = StructType([
    StructField("id",IntegerType(),False),
    StructField("student",StringType(),False)
])
data = [
( 1  , 'Abbot'  ) ,
( 2  , 'Doris'  ) ,
( 3  , 'Emerson') ,
( 4  , 'Green'  ) ,
( 5  , 'Jeames' ) ]
students = spark.createDataFrame(data,schema)
students.show()

+---+-------+
| id|student|
+---+-------+
|  1|  Abbot|
|  2|  Doris|
|  3|Emerson|
|  4|  Green|
|  5| Jeames|
+---+-------+



In [0]:
# Write a solution to swap the seat id of every two consecutive students. If the number of students is odd, the id of the last student is not swapped.
# Return the result table ordered by id in ascending order.
window_spec = Window.orderBy("id")
students.withColumn("new_sitting",\
    when(col("id")%2==1,lead("student",1).over(window_spec))\
    .otherwise(lag("student",1).over(window_spec)))\
.withColumn("new_sitting",\
    when(col("new_sitting").isNull(),col("student"))\
    .otherwise(col("new_sitting")))\
.select("id","new_sitting").show()

+---+-----------+
| id|new_sitting|
+---+-----------+
|  1|      Doris|
|  2|      Abbot|
|  3|      Green|
|  4|    Emerson|
|  5|     Jeames|
+---+-----------+



In [0]:
window_spec = Window.orderBy("id")
students.withColumn("new_sitting", \
    when( col("id")%2==1, \
        when(lead("student",1).over(window_spec).isNull(),col("student"))\
        .otherwise(lead("student",1).over(window_spec)) )\
    .otherwise(lag("student",1).over(window_spec))  )\
.select("id","new_sitting").show()

+---+-----------+
| id|new_sitting|
+---+-----------+
|  1|      Doris|
|  2|      Abbot|
|  3|      Green|
|  4|    Emerson|
|  5|     Jeames|
+---+-----------+



In [0]:
students.createOrReplaceTempView("s")
spark.sql("select id, case when id%2=1 and id<>(select max(id) from s) then lead(student,1) over(order by id) when id%2=0 then lag(student,1) over(order by id) else student end as student from s").show()

+---+-------+
| id|student|
+---+-------+
|  1|  Doris|
|  2|  Abbot|
|  3|  Green|
|  4|Emerson|
|  5| Jeames|
+---+-------+



In [0]:
spark.stop()